In [3]:
import os
import pickle
import numpy
from music21 import note, chord
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

In [58]:
section = 'MIDI\\'
music_name = "~children.mid"
run_folder = r"C:\Users\ppawa\Pranav\Study\Coding\Generative Deep Learning\Projects\Music-Generator\The_Magic_of_MIDI\{}".format(section)
run_folder += '_'.join(music_name)


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join(music_name)

if not os.path.exists(run_folder):
    os.mkdir(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))

mode = 'build'

intervals = range(1)
seq_len = 32

embed_size = 100
rnn_units = 256
use_attention = True

In [59]:
run_folder

'C:\\Users\\ppawa\\Pranav\\Study\\Coding\\Generative Deep Learning\\Projects\\Music-Generator\\The_Magic_of_MIDI\\MIDI\\~_c_h_i_l_d_r_e_n_._m_i_d'

In [60]:
def get_music_list(file_path):
    with open(file_path, 'r') as file:
        music_list = file.read().splitlines()
    return music_list

In [62]:
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0]* seq_len)

            for element in score.flat:
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)
    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '~children.mid'